In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import shap

# 机器学习库
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sklearn.metrics import adjusted_rand_score, silhouette_score

# 层次聚类库
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

# ==============================================================================
# 环境配置
# ==============================================================================
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 150

output_dir = Path('Result')
output_dir.mkdir(parents=True, exist_ok=True)
print(f"所有结果将保存到目录: {output_dir}")

# ==============================================================================
# 数据加载
# ==============================================================================
data_dir = Path('../../Data/0')

# 【文件名修正】将 'X_quantified.csv' 和 'y_quantified.csv' 替换为您的实际文件名
X = pd.read_csv(data_dir / '特征.csv')
y = pd.read_csv(data_dir / '标签.csv').iloc[:, 0]
df_cleaned = pd.read_excel(data_dir / '处理后的数据.xlsx', sheet_name='已分类清洗后数据')
print("数据加载成功。")

chemical_columns = [col for col in df_cleaned.columns if col not in ['文物编号', '文物采样点', '类型', '纹饰', '颜色', '表面风化', '成分总和']]
print(f"识别出的化学成分共 {len(chemical_columns)} 种。")

所有结果将保存到目录: Result
数据加载成功。
识别出的化学成分共 14 种。


In [18]:
# ==============================================================================
# 第1.5步：探索性数据分析 (EDA)
# ==============================================================================
print("\n" + "="*60)
print("             模块一：探索性数据分析 (EDA)")
print("="*60)

# --- 1. 计算描述性统计数据 ---
# 按“类型”分组，然后计算每个化学成分的关键统计指标（均值、标准差、中位数）
descriptive_stats = df_cleaned.groupby('类型')[chemical_columns].agg(['mean', 'std', 'median'])

# 为了更清晰的展示，我们将多层列索引转换为单层
descriptive_stats.columns = ['_'.join(col).strip() for col in descriptive_stats.columns.values]

# 保存统计结果到Excel
stats_path = output_dir / '描述性统计分析.xlsx'
descriptive_stats.to_excel(stats_path)

print("\n--- 描述性统计分析 ---")
print("已计算两种类型玻璃各化学成分的均值、标准差和中位数。")
print(f"详细结果已保存至: {stats_path}")
# 打印部分核心指标的结果
print("\n部分核心指标预览 (均值):")
print(descriptive_stats[['二氧化硅(SiO2)_mean', '氧化钾(K2O)_mean', '氧化铅(PbO)_mean', '氧化钡(BaO)_mean']])


# --- 2. 绘制并列箱线图进行可视化对比 ---
# 为了方便绘图，首先使用pd.melt将宽格式数据转换为长格式
df_melted = df_cleaned.melt(id_vars=['类型'], 
                            value_vars=chemical_columns, 
                            var_name='化学成分', 
                            value_name='含量')

plt.figure(figsize=(18, 10))
# 【写法替换】使用Matplotlib绘制箱线图
# 我们需要为每个化学成分和每个类别准备数据
unique_chems = df_cleaned[chemical_columns].columns
data_to_plot = []
labels = []
for chem in unique_chems:
    # 提取高钾和铅钡的数据
    gaojia_data = df_cleaned[df_cleaned['类型'] == '高钾'][chem].dropna()
    qianbei_data = df_cleaned[df_cleaned['类型'] == '铅钡'][chem].dropna()
    data_to_plot.append(gaojia_data)
    data_to_plot.append(qianbei_data)
    # 添加标签，用换行符来区分，但这里为了整洁我们只标化学成分
    # 后面通过图例来区分

# 创建箱线图
box_plot = plt.boxplot(data_to_plot, patch_artist=True, vert=False)

# 设置颜色
colors = ['lightblue', 'lightgreen'] * len(unique_chems)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)

# 设置Y轴标签
plt.yticks(np.arange(len(unique_chems)) * 2 + 1.5, unique_chems)
plt.gca().invert_yaxis() # 让第一个化学成分在最上面

plt.title('高钾玻璃与铅钡玻璃化学成分含量对比', fontsize=20)
plt.xlabel('含量 (%)', fontsize=14)
plt.ylabel('化学成分', fontsize=14)
plt.grid(axis='x')
# 添加图例
import matplotlib.patches as mpatches
legend_patches = [mpatches.Patch(color='lightblue', label='高钾玻璃'),
                  mpatches.Patch(color='lightgreen', label='铅钡玻璃')]
plt.legend(handles=legend_patches, fontsize=12)

plt.tight_layout()

boxplot_path = output_dir / 'EDA_并列箱线图.png'
plt.savefig(boxplot_path, dpi=300)
plt.close()

print("\n--- 可视化对比 ---")
print("已生成并列箱线图，直观对比两种类型玻璃的化学成分分布。")
print(f"图像已保存至: {boxplot_path}")
print("="*60)


             模块一：探索性数据分析 (EDA)

--- 描述性统计分析 ---
已计算两种类型玻璃各化学成分的均值、标准差和中位数。
详细结果已保存至: Result\描述性统计分析.xlsx

部分核心指标预览 (均值):
    二氧化硅(SiO2)_mean  氧化钾(K2O)_mean  氧化铅(PbO)_mean  氧化钡(BaO)_mean
类型                                                              
铅钡        38.875510       0.173469      33.349184      10.490408
高钾        76.643889       6.401667       0.274444       0.398889

--- 可视化对比 ---
已生成并列箱线图，直观对比两种类型玻璃的化学成分分布。
图像已保存至: Result\EDA_并列箱线图.png


In [19]:
# ==============================================================================
# 第2步：SVM模型训练与准确率评估
# ==============================================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm_rbf = SVC(kernel='rbf', probability=True, random_state=42)
svm_linear = SVC(kernel='linear', random_state=42)

cv_scores_rbf = cross_val_score(svm_rbf, X_scaled, y, cv=10, scoring='accuracy')
cv_scores_linear = cross_val_score(svm_linear, X_scaled, y, cv=10, scoring='accuracy')

svm_rbf.fit(X_scaled, y)
svm_linear.fit(X_scaled, y)
print("RBF核与线性核SVM模型均已训练完成。")

print("\n" + "="*60)
print("             SVM 模型 10折交叉验证性能评估")
print("="*60)
print("\n--- (1) RBF 核 SVM (用于SHAP解释) ---")
print(f"平均准确率: {cv_scores_rbf.mean():.4f} (+/- {cv_scores_rbf.std() * 2:.4f})")
print("\n--- (2) Linear 核 SVM (用于权重分析) ---")
print(f"平均准确率: {cv_scores_linear.mean():.4f} (+/- {cv_scores_linear.std() * 2:.4f})")
print("="*60)

RBF核与线性核SVM模型均已训练完成。

             SVM 模型 10折交叉验证性能评估

--- (1) RBF 核 SVM (用于SHAP解释) ---
平均准确率: 0.9690 (+/- 0.1243)

--- (2) Linear 核 SVM (用于权重分析) ---
平均准确率: 0.9714 (+/- 0.1143)


In [20]:
# ==============================================================================
# 第3步：模型解释 (线性权重 与 SHAP)
# ==============================================================================
# --- (A) 线性SVM特征权重可视化 ---
coefficients = svm_linear.coef_[0]
coef_df = pd.DataFrame({'特征': X.columns, '权重系数': coefficients}).sort_values('权重系数', ascending=False)
top_features_vis = pd.concat([coef_df.head(10), coef_df.tail(10)]).sort_values('权重系数', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
colors = ['red' if c < 0 else 'blue' for c in top_features_vis['权重系数']]
ax.barh(top_features_vis['特征'], top_features_vis['权重系数'], color=colors)
ax.set_title('线性SVM模型特征权重', fontsize=16)
ax.set_xlabel('权重系数 (正向->高钾, 负向->铅钡)', fontsize=12)
ax.grid(axis='x')
plt.tight_layout()
coef_plot_path = output_dir / '线性SVM特征权重.png'
plt.savefig(coef_plot_path, dpi=300)
plt.close()
print(f"线性SVM特征权重图已保存至: {coef_plot_path}")

# --- (B) RBF核SVM的SHAP分析 ---
print("正在进行SHAP分析...")
background_data = shap.kmeans(X_scaled, 50)
explainer = shap.KernelExplainer(lambda x: svm_rbf.predict_proba(x)[:, 1], background_data)
shap_values = explainer.shap_values(X_scaled)

plt.figure()
shap.summary_plot(shap_values, X, feature_names=X.columns, show=False, cmap='cool')
plt.title('SHAP摘要图 (对"高钾玻璃"预测概率的贡献)', fontsize=16)
plt.tight_layout()
summary_plot_path = output_dir / 'SHAP摘要图.png'
plt.savefig(summary_plot_path, dpi=300)
plt.close()
print(f"SHAP摘要图已保存至: {summary_plot_path}")

线性SVM特征权重图已保存至: Result\线性SVM特征权重.png
正在进行SHAP分析...


100%|██████████| 67/67 [00:30<00:00,  2.19it/s]


SHAP摘要图已保存至: Result\SHAP摘要图.png


In [21]:
from sklearn.model_selection import KFold

# ==============================================================================
# 第3.5步：【全新】SVM权重稳定性分析 (Matplotlib版)
# ==============================================================================
print("\n" + "="*60)
print("             模块三：SVM权重稳定性（敏感性）分析")
print("="*60)

# 假设 X, y, X_scaled, output_dir 等变量已在前序步骤中定义

# 我们从之前的分析中选择几个最重要的特征进行观察
# 注意：这里的列表需要与您X矩阵中的实际列名完全对应
key_features_encoded = [
    '氧化铅(PbO)', '氧化钾(K2O)', '二氧化硅(SiO2)', '氧化钡(BaO)',
    '颜色_蓝绿', '纹饰_A' 
]
# 筛选出实际存在的特征
actual_key_features = [f for f in key_features_encoded if f in X.columns]
key_feature_indices = [X.columns.get_loc(f) for f in actual_key_features]

weights_across_folds = []
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 手动执行交叉验证循环以获取每个模型
for train_index, test_index in kf.split(X_scaled):
    X_train, y_train = X_scaled[train_index], y.iloc[train_index]
    
    # 每次都创建一个新的模型实例
    linear_svm_model = SVC(kernel='linear', random_state=42)
    linear_svm_model.fit(X_train, y_train)
    
    fold_coefs = linear_svm_model.coef_[0]
    weights_across_folds.append(fold_coefs[key_feature_indices])

# 将结果转换为DataFrame，便于后续绘图
weights_df = pd.DataFrame(weights_across_folds, columns=actual_key_features)

# ---【写法替换】 使用Matplotlib绘制箱线图 ---
fig, ax = plt.subplots(figsize=(12, 7))
ax.boxplot(weights_df.values, vert=True, labels=weights_df.columns, patch_artist=True)
ax.set_title('关键特征权重在10折交叉验证中的稳定性', fontsize=16)
ax.set_xlabel('特征名称', fontsize=12)
ax.set_ylabel('权重系数值', fontsize=12)
ax.axhline(y=0, color='r', linestyle='--') # 添加y=0的参考线
plt.xticks(rotation=15)
ax.grid(axis='y')
plt.tight_layout()

weights_stability_path = output_dir / 'SVM权重稳定性分析.png'
plt.savefig(weights_stability_path, dpi=300)
plt.close()

print("已完成SVM权重稳定性分析。")
print(f"权重分布箱线图已保存至: {weights_stability_path}")
print("\n各关键特征在10次训练中的权重分布统计:")
print(weights_df.describe())
print("="*60)


             模块三：SVM权重稳定性（敏感性）分析


C:\Users\86185\AppData\Local\Temp\ipykernel_34320\773265586.py:41: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(weights_df.values, vert=True, labels=weights_df.columns, patch_artist=True)


已完成SVM权重稳定性分析。
权重分布箱线图已保存至: Result\SVM权重稳定性分析.png

各关键特征在10次训练中的权重分布统计:
        氧化铅(PbO)   氧化钾(K2O)  二氧化硅(SiO2)   氧化钡(BaO)      颜色_蓝绿
count  10.000000  10.000000   10.000000  10.000000  10.000000
mean   -0.273717   0.339141    0.192281  -0.185155   0.288698
std     0.028645   0.025973    0.031399   0.029732   0.026154
min    -0.295238   0.307951    0.132059  -0.208855   0.240826
25%    -0.291007   0.334576    0.188397  -0.201875   0.283088
50%    -0.290756   0.337161    0.208550  -0.201462   0.289225
75%    -0.267697   0.339041    0.209784  -0.179929   0.292795
max    -0.221645   0.405553    0.214160  -0.116683   0.341518


In [22]:
# ==============================================================================
# 第4步：亚类划分的特征选择
# ==============================================================================
N_FEATURES_TO_SELECT = 8
df_gaojia = df_cleaned[df_cleaned['类型'] == '高钾'].copy()
df_qianbei = df_cleaned[df_cleaned['类型'] == '铅钡'].copy()
selected_features = {}

print("\n" + "="*60)
print("             亚类划分的特征选择过程")
print("="*60)

for glass_type, df_subset in [('高钾', df_gaojia), ('铅钡', df_qianbei)]:
    variances = df_subset[chemical_columns].var().sort_values(ascending=False)
    top_n_features = variances.head(N_FEATURES_TO_SELECT).index.tolist()
    selected_features[glass_type] = top_n_features
    
    print(f"\n--- {glass_type}玻璃 特征选择结果 (基于类内方差) ---")
    print(top_n_features)
    
    data_to_plot = df_subset[top_n_features]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.boxplot(data_to_plot.values, vert=False, labels=data_to_plot.columns, patch_artist=True)
    ax.set_title(f'{glass_type}玻璃 - 候选特征含量分布箱线图', fontsize=16)
    ax.set_xlabel('化学成分含量 (%)', fontsize=12)
    ax.grid(True)
    plt.tight_layout()
    boxplot_path = output_dir / f'{glass_type}玻璃_特征选择箱线图.png'
    plt.savefig(boxplot_path, dpi=300)
    plt.close()
    print(f"特征分布箱线图已保存至: {boxplot_path}")


             亚类划分的特征选择过程

--- 高钾玻璃 特征选择结果 (基于类内方差) ---
['二氧化硅(SiO2)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '五氧化二磷(P2O5)', '氧化钠(Na2O)']


C:\Users\86185\AppData\Local\Temp\ipykernel_34320\2146627436.py:23: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data_to_plot.values, vert=False, labels=data_to_plot.columns, patch_artist=True)


特征分布箱线图已保存至: Result\高钾玻璃_特征选择箱线图.png

--- 铅钡玻璃 特征选择结果 (基于类内方差) ---
['二氧化硅(SiO2)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '二氧化硫(SO2)', '氧化铝(Al2O3)', '氧化铜(CuO)', '氧化钠(Na2O)']


C:\Users\86185\AppData\Local\Temp\ipykernel_34320\2146627436.py:23: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data_to_plot.values, vert=False, labels=data_to_plot.columns, patch_artist=True)


特征分布箱线图已保存至: Result\铅钡玻璃_特征选择箱线图.png


In [23]:
# ==============================================================================
# 第5步：确定最佳亚类数量(K)
# ==============================================================================
optimal_k = {}
for glass_type, df_subset in [('高钾', df_gaojia), ('铅钡', df_qianbei)]:
    features = selected_features[glass_type]
    X_subset_scaled = StandardScaler().fit_transform(df_subset[features])
    linked = linkage(X_subset_scaled, method='ward')
    
    # 绘制树状图
    plt.figure(figsize=(12, 7))
    dendrogram(linked, orientation='top', labels=df_subset['文物编号'].values, distance_sort='descending')
    plt.title(f'{glass_type}玻璃 - 层次聚类树状图', fontsize=16)
    plt.xlabel('文物编号', fontsize=12)
    plt.ylabel('距离 (Ward)', fontsize=12)
    plt.grid(axis='y')
    dendrogram_path = output_dir / f'{glass_type}玻璃_层次聚类树状图.png'
    plt.savefig(dendrogram_path, dpi=300)
    plt.close()
    print(f"\n{glass_type}玻璃的树状图已保存至: {dendrogram_path}")

    # 计算轮廓系数以确定最佳K
    k_range = range(2, 9)
    scores = []
    for k in k_range:
        labels = fcluster(linked, k, criterion='maxclust')
        if len(set(labels)) > 1:
            scores.append(silhouette_score(X_subset_scaled, labels))
        else:
            scores.append(-1) # 无法计算，给一个无效分
            
    best_k = k_range[np.argmax(scores)]
    optimal_k[glass_type] = best_k
    
    # 绘制K值-轮廓系数图
    plt.figure(figsize=(8, 5))
    plt.plot(k_range, scores, marker='o')
    plt.title(f'{glass_type}玻璃 - 轮廓系数法确定最佳K值', fontsize=16)
    plt.xlabel('亚类数量 (K)', fontsize=12)
    plt.ylabel('轮廓系数', fontsize=12)
    plt.grid(True)
    plt.axvline(x=best_k, color='r', linestyle='--', label=f'最佳 K = {best_k} (得分: {max(scores):.3f})')
    plt.legend()
    k_plot_path = output_dir / f'{glass_type}玻璃_最佳K值选择.png'
    plt.savefig(k_plot_path, dpi=300)
    plt.close()
    print(f"{glass_type}玻璃的最佳K值选择图已保存至: {k_plot_path}")

print(f"\n根据轮廓系数，确定的最佳K值为: {optimal_k}")


高钾玻璃的树状图已保存至: Result\高钾玻璃_层次聚类树状图.png
高钾玻璃的最佳K值选择图已保存至: Result\高钾玻璃_最佳K值选择.png

铅钡玻璃的树状图已保存至: Result\铅钡玻璃_层次聚类树状图.png
铅钡玻璃的最佳K值选择图已保存至: Result\铅钡玻璃_最佳K值选择.png

根据轮廓系数，确定的最佳K值为: {'高钾': 5, '铅钡': 2}


In [24]:
# ==============================================================================
# 第6步：最终亚类划分与可视化
# ==============================================================================
all_clustered_data = []

for glass_type, df_subset in [('高钾', df_gaojia), ('铅钡', df_qianbei)]:
    features = selected_features[glass_type]
    k = optimal_k[glass_type]
    X_subset_scaled = StandardScaler().fit_transform(df_subset[features])
    linked = linkage(X_subset_scaled, method='ward')
    df_subset['亚类'] = fcluster(linked, k, criterion='maxclust')
    
    # PCA降维可视化
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_subset_scaled)
    df_subset[['PCA1', 'PCA2']] = X_pca
    
    fig, ax = plt.subplots(figsize=(8, 6))
    unique_labels = sorted(df_subset['亚类'].unique())
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))
    for label, color in zip(unique_labels, colors):
        subset_to_plot = df_subset[df_subset['亚类'] == label]
        ax.scatter(subset_to_plot['PCA1'], subset_to_plot['PCA2'], c=[color], label=f'亚类 {label}', s=80, alpha=0.8)
    ax.set_title(f'{glass_type}玻璃亚类 - PCA降维图 (K={k})', fontsize=16)
    ax.legend(title='亚类')
    ax.grid(True)
    pca_path = output_dir / f'{glass_type}玻璃_亚类PCA图.png'
    plt.savefig(pca_path, dpi=300)
    plt.close()

    # 带编号的热力图
    df_sorted = df_subset.sort_values('亚类')
    X_sorted_scaled = StandardScaler().fit_transform(df_sorted[features])
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(X_sorted_scaled, cmap='magma', aspect='auto')
    ax.set_xticks(np.arange(len(features)), labels=features, rotation=45, ha='right')
    ax.set_yticks(np.arange(len(df_sorted)), labels=df_sorted['文物编号'].values, fontsize=8)
    
    if len(unique_labels) > 1:
        cluster_boundaries = np.where(df_sorted['亚类'].diff())[0]
        for boundary in cluster_boundaries:
            ax.axhline(y=boundary - 0.5, color='lime', linewidth=2.5)
            
    ax.set_title(f'{glass_type}玻璃亚类化学成分热力图 (K={k})', fontsize=16)
    fig.colorbar(im, ax=ax, label='标准化含量', shrink=0.7)
    heatmap_path = output_dir / f'{glass_type}玻璃_亚类热力图_带编号.png'
    plt.savefig(heatmap_path, dpi=300)
    plt.close()
    
    all_clustered_data.append(df_sorted)

output_clustered_file = output_dir / '最终亚类划分结果.xlsx'
with pd.ExcelWriter(output_clustered_file) as writer:
    all_clustered_data[0].to_excel(writer, sheet_name='高钾玻璃亚类', index=False)
    all_clustered_data[1].to_excel(writer, sheet_name='铅钡玻璃亚类', index=False)

print(f"\n最终亚类划分可视化图像及Excel文件已保存。")


最终亚类划分可视化图像及Excel文件已保存。


In [26]:
from sklearn.utils import resample
from sklearn.metrics import adjusted_rand_score
import numpy as np
import pandas as pd

# ==============================================================================
# 【最终版】层次聚类灵敏度分析 (基于子采样法)
# ==============================================================================
# 参数定义
N_ITERATIONS = 10000      # 重复子采样的次数 k
N_SAMPLES_TO_REMOVE = 10 # 每次随机移除的样本数量

# 假设 all_clustered_data 和 selected_features 变量已在前序步骤中定义
# all_clustered_data[0] -> 高钾玻璃
# all_clustered_data[1] -> 铅钡玻璃
df_gaojia_clustered = all_clustered_data[0]
df_qianbei_clustered = all_clustered_data[1]

ari_scores_subsample = {}

print("\n" + "="*60)
print("     最终灵敏度分析 (基于子采样法 + ARI)")
print("="*60)

for glass_type, df_full_data in [('高钾', df_gaojia_clustered), ('铅钡', df_qianbei_clustered)]:
    print(f"\n正在对 {glass_type} 玻璃进行灵敏度分析...")
    
    features = selected_features[glass_type]
    k_clusters = df_full_data['亚类'].nunique() # 从之前的聚类结果中获取K值
    
    # C_full: 在D_full上进行层次聚类得到的基准划分结果
    baseline_labels = df_full_data['亚类']
    
    scores_per_iteration = []
    
    for i in range(N_ITERATIONS):
        # 1. 创建子数据集 D_sub by removing 10 samples
        n_total = len(df_full_data)
        n_subsample = n_total - N_SAMPLES_TO_REMOVE
        
        # 随机选择 n_subsample 个索引 (无放回)
        subsample_indices = np.random.choice(df_full_data.index, size=n_subsample, replace=False)
        df_subsample = df_full_data.loc[subsample_indices]
        
        # 2. 在子数据集上重新进行层次聚类，得到 C_sub
        X_subsample_scaled = StandardScaler().fit_transform(df_subsample[features])
        
        if len(df_subsample) < k_clusters: continue # 样本太少无法聚类

        linked_subsample = linkage(X_subsample_scaled, method='ward')
        subsample_labels = fcluster(linked_subsample, k_clusters, criterion='maxclust')
        
        # 3. 提取过滤后的基准划分 C'_full
        # 即，从原始划分中，只取出子样本对应的那些标签
        baseline_labels_filtered = baseline_labels.loc[subsample_indices]
        
        # 4. 计算 ARI_i
        # 确保两个划分中的簇数量都大于1
        if len(np.unique(baseline_labels_filtered)) < 2 or len(np.unique(subsample_labels)) < 2:
            continue # ARI无法计算（如果所有点都被分到一类）
            
        ari_i = adjusted_rand_score(baseline_labels_filtered, subsample_labels)
        scores_per_iteration.append(ari_i)

    ari_scores_subsample[glass_type] = scores_per_iteration
    
    # 5. 计算最终的平均子样本稳定性得分 (S_bar_ARI)
    s_bar_ari = np.mean(scores_per_iteration)
    print(f"{glass_type} 玻璃的平均子样本稳定性得分 (S_bar_ARI) 为: {s_bar_ari:.4f}")

# 6. 可视化ARI分数分布
fig, ax = plt.subplots(figsize=(8, 6))
ax.boxplot(ari_scores_subsample.values(), vert=True, labels=ari_scores_subsample.keys(), patch_artist=True)
ax.set_title('聚类稳定性评估 (子采样法)', fontsize=16)
ax.set_ylabel('调整兰德指数 (ARI)', fontsize=12)
ax.set_xlabel('玻璃类别', fontsize=12)
ax.yaxis.grid(True)
ax.set_ylim(-0.1, 1.1)
means = [np.mean(s) for s in ari_scores_subsample.values()]
ax.scatter(range(1, len(means) + 1), means, marker='o', color='red', s=50, zorder=3, label=f'平均ARI ($\\bar{{S}}_{{ARI}}$)')
ax.legend()

plt.tight_layout()
ari_plot_path = output_dir / '最终灵敏度分析_子采样ARI箱线图.png'
plt.savefig(ari_plot_path, dpi=300)
plt.close()

print(f"\n灵敏度分析结果箱线图已保存至: {ari_plot_path}")


     最终灵敏度分析 (基于子采样法 + ARI)

正在对 高钾 玻璃进行灵敏度分析...
高钾 玻璃的平均子样本稳定性得分 (S_bar_ARI) 为: 0.6741

正在对 铅钡 玻璃进行灵敏度分析...
铅钡 玻璃的平均子样本稳定性得分 (S_bar_ARI) 为: 0.2374


C:\Users\86185\AppData\Local\Temp\ipykernel_34320\4205097887.py:73: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(ari_scores_subsample.values(), vert=True, labels=ari_scores_subsample.keys(), patch_artist=True)



灵敏度分析结果箱线图已保存至: Result\最终灵敏度分析_子采样ARI箱线图.png
